![](path)

![](/Volumes/jpm/naval/csv/Screenshot 2025-02-25 at 5.15.48 PM.png)

In [0]:
%sql
use catalog jpm;
create schema if not exists bharat_bronze;
create schema if not exists bharat_silver;
create schema if not exists bharat_gold;

In [0]:
df_bronze=spark.read.csv("/Volumes/jpm/subro/financial/Financial Sample.csv", header=True,inferSchema=True).toDF(*[c.replace(' ', '_').lower() for c in spark.read.format('csv').option('header', 'true').load('/Volumes/jpm/subro/financial/Financial Sample.csv').columns])
df_bronze.write.mode("overwrite").saveAsTable("jpm.bharat_bronze.bharat_financial_data_bronze_3")

In [0]:
df_bronze.createOrReplaceTempView("bharat_financial_data_bronze_temp")
#%sql Select * from bharat_financial_data_bronze_3


In [0]:
%sql
CREATE or Replace table jpm.bharat_bronze.financial_bronze as 
(SELECT `segment` as Segment,
       `country` as Country,
       `product` as Product,
       `discount_band` as Discount_Band,
       `units_sold` as Units_Sold,
       `manufacturing_price` as Manufacturing_Price,
       `sale_price` as Sale_Price,
       `gross_sales` as Gross_Sales,
       `discounts` as Discounts,
       `sales` as Sales,
       `cogs` as COGS,
       `profit` as Profit,
       `date` as Date,
       `month_number` as Month_Number,
       `month_name` as Month_Name,
       `year` as Year,
       current_date() as Ingestion_Date
FROM bharat_financial_data_bronze_temp)

In [0]:
%sql
select * from jpm.bharat_bronze.financial_bronze

In [0]:
%sql
SELECT 
    Segment,
    Country,
    Product,
    Discount_Band,
    REPLACE(Units_Sold, '$', '') AS Units_Sold,
    Manufacturing_Price,
    Sale_Price,
    Gross_Sales,
    Discounts,
    Sales,
    COGS,
    Profit,
    Date,
    Month_Name,
    Month_Number,
    Year,
    ingestion_date
FROM jpm.bharat_bronze.financial_bronze

from pyspark.sql.functions import regexp_replace, col

# Remove spaces from all string columns
df_cleaned = df1.select([regexp_replace(col(c), r'\s+', '').alias(c) if c != 'units_sold' else col(c) for c in df1.columns])

# Remove $ symbol from 'units_sold' column
df_cleaned = df_cleaned.withColumn('units_sold', regexp_replace(col('units_sold'), r'\$', ''))

display(df_cleaned)